In [27]:
import numpy as np
import pandas as pd
from datetime import datetime


In [28]:
import warnings

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Audio Inference Data
0: Silent

1: Voice

2: Noise


In [29]:
AudioData = pd.DataFrame()
AudioData = pd.read_csv('../dataset/Merged/allAudioRaw.csv')
print(AudioData)

           timestamp   audio inference userId
0         1364356800                 0    u59
1         1364356801                 0    u59
2         1364356802                 0    u59
3         1364356803                 0    u59
4         1364356804                 0    u59
...              ...               ...    ...
99298218  1369943987                 0    u54
99298219  1369943988                 0    u54
99298220  1369943989                 0    u54
99298221  1369943990                 0    u54
99298222  1369943991                 0    u54

[99298223 rows x 3 columns]


In [30]:
AudioData = AudioData.sort_values(by='userId')

In [31]:
AudioData['timestamp'] = pd.to_datetime(AudioData['timestamp'], unit = 's')
AudioData.rename(columns={' audio inference': 'audioInference'}, inplace=True)
AudioData['date'] = AudioData['timestamp'].dt.date
AudioData['time'] = AudioData['timestamp'].dt.time
AudioData['date'] = pd.to_datetime(AudioData['date'])
AudioData['week'] = AudioData['date'].dt.isocalendar().week

In [32]:
print(AudioData)

                   timestamp  audioInference userId       date      time  week
24960596 2013-05-11 16:56:42               1    u00 2013-05-11  16:56:42    19
23930311 2013-04-16 08:41:55               0    u00 2013-04-16  08:41:55    16
23930310 2013-04-16 08:41:54               0    u00 2013-04-16  08:41:54    16
23930309 2013-04-16 08:41:53               0    u00 2013-04-16  08:41:53    16
23930308 2013-04-16 08:41:52               0    u00 2013-04-16  08:41:52    16
...                      ...             ...    ...        ...       ...   ...
3122755  2013-05-12 15:48:53               2    u59 2013-05-12  15:48:53    19
3122756  2013-05-12 15:48:55               0    u59 2013-05-12  15:48:55    19
3122757  2013-05-12 15:48:55               0    u59 2013-05-12  15:48:55    19
3122751  2013-05-12 15:48:50               0    u59 2013-05-12  15:48:50    19
0        2013-03-27 04:00:00               0    u59 2013-03-27  04:00:00    13

[99298223 rows x 6 columns]


In [33]:
# Convert 'time' column to datetime format
AudioData['hour'] = AudioData['timestamp'].dt.hour
# print(ActivityData)

bins = [-1, 6, 12, 18, 24]
labels = ['EarlyMorning', 'Morning', 'Afternoon', 'Evening']
AudioData['timeCategory'] = pd.cut(AudioData['hour'], bins=bins, labels=labels, right=False)
AudioData.drop(columns=['hour'], inplace=True)

print(AudioData)

                   timestamp  audioInference userId       date      time  \
24960596 2013-05-11 16:56:42               1    u00 2013-05-11  16:56:42   
23930311 2013-04-16 08:41:55               0    u00 2013-04-16  08:41:55   
23930310 2013-04-16 08:41:54               0    u00 2013-04-16  08:41:54   
23930309 2013-04-16 08:41:53               0    u00 2013-04-16  08:41:53   
23930308 2013-04-16 08:41:52               0    u00 2013-04-16  08:41:52   
...                      ...             ...    ...        ...       ...   
3122755  2013-05-12 15:48:53               2    u59 2013-05-12  15:48:53   
3122756  2013-05-12 15:48:55               0    u59 2013-05-12  15:48:55   
3122757  2013-05-12 15:48:55               0    u59 2013-05-12  15:48:55   
3122751  2013-05-12 15:48:50               0    u59 2013-05-12  15:48:50   
0        2013-03-27 04:00:00               0    u59 2013-03-27  04:00:00   

          week  timeCategory  
24960596    19     Afternoon  
23930311    16       Morn

In [34]:
AudioData = AudioData.sort_values(by='week')

In [35]:
weeks = AudioData['week'].unique()
# print(weeks)
# print(len(weeks))
# print(weeks[9])
dfs = []

for week in weeks[:-1]:
    twoWeekData = AudioData[(AudioData['week'] == week) | (AudioData['week'] == week+1)]
    # print(twoWeekData['week'].unique())
    grouped = twoWeekData.groupby(['userId','timeCategory', 'audioInference']).size().reset_index(name='count')
    # print(grouped)
    # Pivot the grouped DataFrame
    pivot_df = grouped.pivot_table(index='userId', columns=['timeCategory', 'audioInference'], values='count').fillna(0)
    print(pivot_df)

    #  Concatenate 'timeCategory' and 'activityInference' to create column names
    pivot_df.columns = [f"Audio{category}{inference}" for category, inference in pivot_df.columns]

    # Reset index to make 'userId' a column
    pivot_df.reset_index(inplace=True)
    pivot_df['WeekId']= week - weeks[0]
    dfs.append(pivot_df)

result_df = pd.concat(dfs)
result_df.reset_index(inplace=True)


timeCategory   EarlyMorning                    Morning                    \
audioInference            0        1        2        0        1        2   
userId                                                                     
u00                 54484.0  42088.0  18262.0  60296.0   8178.0   7482.0   
u01                 55375.0  76962.0  33123.0  65776.0   5302.0   4202.0   
u02                 57214.0  23547.0  18661.0  67229.0   1663.0    112.0   
u03                 59007.0   2282.0   1353.0  67158.0    111.0    119.0   
u04                 51336.0  25755.0  25512.0  58332.0  11875.0  17558.0   
u05                 55419.0  31516.0  24172.0  65483.0    106.0   1631.0   
u07                 55241.0  46078.0  33992.0  62798.0  12448.0  12293.0   
u08                 51567.0  51561.0  31539.0  65844.0   2163.0   2449.0   
u09                 50450.0  59097.0  20451.0  47906.0   2216.0   3676.0   
u10                 62611.0  17216.0  12153.0  66982.0    129.0     89.0   
u12         

In [36]:
result_df = result_df.drop(columns=['index'])
print(result_df)

    userId  AudioEarlyMorning0  AudioEarlyMorning1  AudioEarlyMorning2  \
0      u00             54484.0             42088.0             18262.0   
1      u01             55375.0             76962.0             33123.0   
2      u02             57214.0             23547.0             18661.0   
3      u03             59007.0              2282.0              1353.0   
4      u04             51336.0             25755.0             25512.0   
..     ...                 ...                 ...                 ...   
426    u54             10603.0              9710.0              4781.0   
427    u56             60574.0             25537.0             28762.0   
428    u57             50390.0             65704.0             46693.0   
429    u58             39449.0             61569.0             29227.0   
430    u59            140549.0             42769.0             83108.0   

     AudioMorning0  AudioMorning1  AudioMorning2  AudioAfternoon0  \
0          60296.0         8178.0         

In [37]:
result_df.to_csv('../dataset/BasicFeatures/audioCounts.csv', index=False)